In [1]:
import os
import torch
import pandas as pd
from torchvision import transforms, datasets
from PIL import Image
from glob import glob
from efficientnet_pytorch import EfficientNet
from torch.utils.data import DataLoader

# 장비 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. 이미지 데이터 불러오기 및 전처리

class ResizeIfNeeded:
    def __call__(self, image):
        # 이미지의 가로, 세로 길이를 확인
        width, height = image.size
        
        # 가로 또는 세로 길이가 224 미만인 경우를 확인하여 크기를 늘림
        new_width = width if width >= 224 else 224
        new_height = height if height >= 224 else 224
        
        # 이미지 크기를 변경
        resized_image = image.resize((new_width, new_height), Image.BICUBIC)
        return resized_image

transform = transforms.Compose([
    # 가로 또는 세로가 224 미만일 경우 크기를 늘림
    transforms.Lambda(ResizeIfNeeded()),
    # 가로와 세로가 모두 224 이상일 경우 224x224로 랜덤 크롭
    transforms.RandomCrop((224, 224)),
    # 이미지를 PyTorch 텐서로 변환
    transforms.ToTensor(),
])

# ImageFolder 사용
dataset = datasets.ImageFolder(root='./test_data', transform=transform)
batch_size = 256  # 원하는 배치 크기를 설정
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# 2. 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
model.load_state_dict(torch.load('./model_state_dict.pth'))
model = model.to(device)  # 장비 할당
model.eval()  # 평가 모드로 전환

Loaded pretrained weights for efficientnet-b0


In [ ]:
# 3. 예측하기
predictions = []

for images, _ in dataloader:  # 레이블은 사용하지 않음
    images = images.to(device)  # 장비 할당
    with torch.no_grad():
        outputs = model(images)
        prediction = outputs.argmax(dim=1).cpu().numpy()
        predictions.extend(prediction)

# 4. 결과 저장
df = pd.read_csv('./test_data/sample_submission.csv')
df['answer'] = predictions
df.to_csv('./test_data/sample_submission.csv', index=False)

In [2]:
df = pd.read_csv('./test_data/sample_submission.csv')
df['answer'] = predictions
df.to_csv('./test_data/sample_submission.csv', index=False)

NameError: name 'pd' is not defined

In [3]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('test_data/sample_submission.csv')

# 'answer' 컬럼의 1과 0을 바꾸기
df['answer'] = 1 - df['answer']

# 변경된 데이터프레임을 다시 CSV 파일로 저장하기
df.to_csv('test_data/sample_submission_modified.csv', index=False)

print("File saved successfully.")

File saved successfully.
